# Compare NLP Techniques: Build Model On doc2vec Vectors

### Read In Cleaned Text

In [1]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')

/tmp/ipykernel_19764/1128796122.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Create doc2vec Vectors

In [2]:
# Created TaggedDocument vectors for each text message in the training and test sets
tagged_docs_train = [gensim.models.doc2vec.TaggedDocument(v, [i])
                     for i, v in enumerate(X_train['clean_text'])]
tagged_docs_test = [gensim.models.doc2vec.TaggedDocument(v, [i])
                    for i, v in enumerate(X_test['clean_text'])]

In [3]:
# What do these TaggedDocument objects look like?
tagged_docs_train[:10]

[TaggedDocument(words="['spare', 'power', 'supplies']", tags=[0]),
 TaggedDocument(words="['urgent', 'trying', 'contact', 'u', 'todays', 'draw', 'shows', 'å', '800', 'prize', 'guaranteed', 'call', '09050001808', 'land', 'line', 'claim', 'm95', 'valid12hrs']", tags=[1]),
 TaggedDocument(words="['1st', 'wk', 'free', 'gr8', 'tones', 'str8', '2', 'u', 'wk', 'txt', 'nokia', '8007', 'classic', 'nokia', 'tones', 'hit', '8007', 'polys', 'nokia150p', 'poly200p', '16']", tags=[2]),
 TaggedDocument(words="['call', 'meet']", tags=[3]),
 TaggedDocument(words="['haha', 'really', 'oh', 'deduct', 'lesson', 'tmr']", tags=[4]),
 TaggedDocument(words="['sounds', 'great', 'home']", tags=[5]),
 TaggedDocument(words="['ì', 'login', 'dat', 'time', 'dad', 'fetching', 'ì', 'home']", tags=[6]),
 TaggedDocument(words="['probably', 'still', 'going', 'stuff']", tags=[7]),
 TaggedDocument(words="['hey', 'guy', 'know', 'breathing', 'neck', 'get', 'bud', 'anyway', 'youd', 'able', 'get', 'half', 'track', 'usf', 'tonig

In [4]:
# Train a basic doc2vec model
d2v_model = gensim.models.Doc2Vec(tagged_docs_train,
                                  vector_size=100,
                                  window=5,
                                  min_count=2)

In [5]:
# Infer the vectors to be used in training and testing
train_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_train]
test_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_test]   

### Fit RandomForestClassifier On Top Of Document Vectors

In [6]:
# Fit a basic model, make predictions on the holdout test set, and the generate the evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

rf = RandomForestClassifier()
rf_model = rf.fit(train_vectors, y_train.values.ravel())

y_pred = rf_model.predict(test_vectors)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.861 / Recall: 0.373 / Accuracy: 0.898
